# SamerAlgorithms : Turning Math into Quantum Power

**SamerAlgorithms** converts complex mathematical expressions into optimization problems, making them solvable on Quantum Annealing hardware (Dynex - D-Wave) with full support for all arithmetic operators, it bridges the gap between classical mathematics and quantum computing by translating equations into BQM/QUBO/Ising formats for any mathematical challenge.

### Load Libraries

In [1]:
from SamerAlgorithms import Expression, fVar, Qubits, DecodeSolution

- **`Expression`**: This decorator is used to define a method that includes all the variables and expressions you want to convert into an optimization format. It serves as the entry point for translating your mathematical expressions into a form suitable for quantum annealing.

- **`fVar`**: This function allows you to specify fixed variables within your equation, providing constraints that the optimization process will respect during the translation into a quantum-ready format.

- **`Qubits`**: This function determines the number of qubits required for each variable in your expression, ensuring the optimization problem is mapped correctly onto the quantum annealing hardware.

- **`DecodeSolution`**: After the quantum annealing process is complete, this function is used to decode the sampleset into a format that is easily interpretable, turning the quantum output into meaningful, presentable data.

--------------------------

## Example 1:

$
\left[ \left( a \times b \right) + c \right] \% d = 0
$

where:
- $a$, $b$, $d$ have 3 qubits (8 possible values [0-7]) 
- $c$ = 6

In [2]:
@Expression
def Minimize(a, b, c, d) -> Qubits(4): 
    a: Qubits(3) # 3 qubits needed for `a` variable
    b: Qubits(3) # 3 qubits needed for `b` variable
    c: fVar[Qubits(3)]  # 3 qubits needed for `c` variable and fVar means its value will be provided later.
    d: Qubits(3) # 3 qubits needed for `d` variable
    Expr = ((a * b) + c) % d # your expression
    return Expr
c = 6 
solve = Minimize.assign(c=c) # assign fVar with its fixed value

#### Translate the Expression into BQM

In [3]:
bqm = solve.BQM()

### Using Dynex to solve the expression (Neuromorphic Quantum Annealer)

In [4]:
import dynex
model = dynex.BQM(bqm)
sampler = dynex.DynexSampler(model, mainnet=True, description='SamerAlgorithms', bnb=False)
sampleset = sampler.sample(num_reads=10000, annealing_time = 200, debugging=False, shots=1, is_cluster=True)

╭────────────┬──────────┬─────────────────┬─────────────┬───────────┬────────────────┬────────────┬─────────┬────────────────╮
│   DYNEXJOB │   QUBITS │   QUANTUM GATES │   BLOCK FEE │   ELAPSED │   WORKERS READ │   CIRCUITS │   STEPS │   GROUND STATE │
├────────────┼──────────┼─────────────────┼─────────────┼───────────┼────────────────┼────────────┼─────────┼────────────────┤
│      32049 │       41 │             496 │        0.00 │      0.13 │              1 │      10000 │     200 │      688000.00 │
╰────────────┴──────────┴─────────────────┴─────────────┴───────────┴────────────────┴────────────┴─────────┴────────────────╯
╭────────────┬─────────────────┬────────────┬───────┬──────────┬───────────────┬─────────────────────────────┬───────────┬──────────╮
│     WORKER │         VERSION │   CIRCUITS │   LOC │   ENERGY │       RUNTIME │                 LAST UPDATE │     STEPS │   STATUS │
├────────────┼─────────────────┼────────────┼───────┼──────────┼───────────────┼─────────────────

#### Decode Sampleset

In [5]:
decoded = DecodeSolution(solve, sampleset)
best = min(decoded, key=lambda x: x.energy)
print(best.sample)

{'a': 7, 'b': 7, 'd': 5}


#### Validate (This should be zero [BALANCED])

In [6]:
a = best.sample['a']
b = best.sample['b']
d = best.sample['d']
Expr = ((a * b) + c) % d
print("Expression Result: ",Expr)

Expression Result:  0


-----------------------------------

## Example 2:

$
\left[ \left( \left( p + q \right) \right) \% r \right] + \left[ \left( \left( p \times r \right) - \left( q + s \right) \right) \% p \right] - \left( s - r \right) + \left[ \left( \left( s \times p \right) + \left( q - r \right) \right) \% 2 \right] = 0
$

where:
- $q$, $r$, $s$ have 3 qubits (8 possible values [0-7]) 
- $p = 1$
- $2$ Moduli

In [2]:
@Expression
def Minimize(p, q, r, s) -> Qubits(4):
    p: fVar[Qubits(2)] # 2 qubits needed for `p` variable and fVar means its value will be provided later.
    q: Qubits(3)       # 3 qubits needed for `q` variable
    r: Qubits(3)       # 3 qubits needed for `r` variable
    s: Qubits(3)       # 3 qubits needed for `s` variable
    Expr = (((p + q)) % r) + (((p * r) - (q + s)) % p) - ((s - r)) + (((s * q) + (q - r)) % 2)
    return Expr
p = 1
solve = Minimize.assign(p=p) # assign fVar with its fixed value

#### Translate the Expression into BQM

In [4]:
bqm = solve.BQM()

### Using Dynex to solve the expression (Neuromorphic Quantum Annealer)

In [6]:
import dynex
model = dynex.BQM(bqm)
sampler = dynex.DynexSampler(model, mainnet=False, description='SamerAlgorithms', bnb=False)
sampleset = sampler.sample(num_reads=10000, annealing_time = 200, debugging=False, shots=1, is_cluster=False)

[DYNEX] SAMPLER INITIALISED
[DYNEX|TESTNET] *** WAITING FOR READS ***
╭────────────┬──────────┬─────────────────┬─────────────┬───────────┬───────────────────────────┬────────────┬─────────┬────────────────╮
│   DYNEXJOB │   QUBITS │   QUANTUM GATES │   BLOCK FEE │ ELAPSED   │ WORKERS READ              │ CIRCUITS   │ STEPS   │ GROUND STATE   │
├────────────┼──────────┼─────────────────┼─────────────┼───────────┼───────────────────────────┼────────────┼─────────┼────────────────┤
│         -1 │       54 │             798 │           0 │           │ *** WAITING FOR READS *** │            │         │                │
╰────────────┴──────────┴─────────────────┴─────────────┴───────────┴───────────────────────────┴────────────┴─────────┴────────────────╯

[DYNEX] FINISHED READ AFTER 0.20 SECONDS
[DYNEX] SAMPLESET READY


#### Decode Sampleset

In [7]:
decoded = DecodeSolution(solve, sampleset)
best = min(decoded, key=lambda x: x.energy)
print(best.sample)

{'q': 7, 'r': 7, 's': 3}


#### Validate (This should be zero [BALANCED])

In [8]:
q = best.sample['q']
r = best.sample['r']
s = best.sample['s']
Expr = (((p + q)) % r) + (((p * r) - (q + s)) % p) - ((s - r)) + (((s * q) + (q - r)) % 2)
print("Expression Result: ", Expr)

Expression Result:  0


--------------------------

## Example 3: (Arithmetic Modular)

$
N \% d = 0
$

where:
- $N$ has 6 qubits
- $d$ has 3 qubits

In [13]:
@Expression
def Minimize(N, d) -> Qubits(6): 
    N: fVar[Qubits(6)]  # 6 qubits needed for `N` variable and fVar means its value will be provided later.
    d: Qubits(3) # 3 qubits needed for `d` variable
    Expr = N % d # your expression
    return Expr
N = 35
solve = Minimize.assign(N=N) # assign fVar with its fixed value

#### Translate the Expression into BQM

In [14]:
bqm = solve.BQM()

### Using Dynex to solve the expression (Neuromorphic Quantum Annealer)

In [15]:
model = dynex.BQM(bqm)
sampler = dynex.DynexSampler(model, mainnet=True, description='SamerAlgorithms', bnb=False)
sampleset = sampler.sample(num_reads=10000, annealing_time = 200, debugging=False, shots=1, is_cluster=True)

╭────────────┬──────────┬─────────────────┬─────────────┬───────────┬────────────────┬────────────┬─────────┬────────────────╮
│   DYNEXJOB │   QUBITS │   QUANTUM GATES │   BLOCK FEE │   ELAPSED │   WORKERS READ │   CIRCUITS │   STEPS │   GROUND STATE │
├────────────┼──────────┼─────────────────┼─────────────┼───────────┼────────────────┼────────────┼─────────┼────────────────┤
│      32051 │       10 │              11 │        0.00 │      0.10 │              1 │      10000 │     200 │       10000.00 │
╰────────────┴──────────┴─────────────────┴─────────────┴───────────┴────────────────┴────────────┴─────────┴────────────────╯
╭────────────┬─────────────────┬────────────┬───────┬──────────┬──────────────┬─────────────────────────────┬───────────┬──────────╮
│     WORKER │         VERSION │   CIRCUITS │   LOC │   ENERGY │      RUNTIME │                 LAST UPDATE │     STEPS │   STATUS │
├────────────┼─────────────────┼────────────┼───────┼──────────┼──────────────┼────────────────────

#### Decode Sampleset

In [16]:
decoded = DecodeSolution(solve, sampleset)
best = min(decoded, key=lambda x: x.energy)
print(best.sample)

{'d': 5}


#### Validate (This should be zero [BALANCED])

In [17]:
d = best.sample['d']
Expr = N % d
print("Expression Result: ", Expr)

Expression Result:  0


-------------------------------------

## Example 4: (Integer Factorization)

$
N - [\left( p \times q \right)] = 0
$

where:
- $p$, $q$ have 3 qubits
- $N$ has 6 qubits

In [18]:
@Expression
def Minimize(N, p, q) -> Qubits(6):
    N: fVar[Qubits(6)] # 6 qubits needed for `N` variable and fVar means its value will be provided later.
    p: Qubits(3) # 3 qubits needed for `p` variable
    q: Qubits(3) # 3 qubits needed for `q` variable
    Expr = N - (p * q)
    return Expr
N = 36
solve = Minimize.assign(N=N)

#### Translate the Expression into BQM

In [19]:
bqm = solve.BQM()

### Using Dynex to solve the expression (Neuromorphic Quantum Annealer)

In [20]:
model = dynex.BQM(bqm)
sampler = dynex.DynexSampler(model, mainnet=True, description='SamerAlgorithms', bnb=False)
sampleset = sampler.sample(num_reads=10000, annealing_time = 200, debugging=False, shots=1, is_cluster=True)

╭────────────┬──────────┬─────────────────┬─────────────┬───────────┬────────────────┬────────────┬─────────┬────────────────╮
│   DYNEXJOB │   QUBITS │   QUANTUM GATES │   BLOCK FEE │   ELAPSED │   WORKERS READ │   CIRCUITS │   STEPS │   GROUND STATE │
├────────────┼──────────┼─────────────────┼─────────────┼───────────┼────────────────┼────────────┼─────────┼────────────────┤
│      32052 │       26 │             167 │        0.00 │      0.10 │              1 │      10000 │     200 │      287000.00 │
╰────────────┴──────────┴─────────────────┴─────────────┴───────────┴────────────────┴────────────┴─────────┴────────────────╯
╭────────────┬─────────────────┬────────────┬───────┬──────────┬──────────────┬─────────────────────────────┬───────────┬──────────╮
│     WORKER │         VERSION │   CIRCUITS │   LOC │   ENERGY │      RUNTIME │                 LAST UPDATE │     STEPS │   STATUS │
├────────────┼─────────────────┼────────────┼───────┼──────────┼──────────────┼────────────────────

#### Decode Sampleset

In [21]:
decoded = DecodeSolution(solve, sampleset)
best = min(decoded, key=lambda x: x.energy)
print(best.sample)

{'p': 6, 'q': 6}


#### Validate (This should be zero [BALANCED])

In [22]:
p = best.sample['p']
q = best.sample['q']
Expr = N - (p * q)
print("Expression Result: ", Expr)

Expression Result:  0
